## Despacho Econômico de Energia

O problema consiste em determinar a distribuição ótima de geração entre as unidades geradoras disponíveis para atender à demanda de energia elétrica ao menor custo possível.

A função objetivo é minimizar o custo total de geração:

$$
\min f(P_1, P_2, P_3) = C_1(P_1) + C_2(P_2) + C_3(P_3)
$$

Onde as curvas de custo são dadas por:

$$
C_1(P_1) = 0.15 P_1^2 + 38 P_1 + 756
$$
$$
C_2(P_2) = 0.1 P_2^2 + 46 P_2 + 451
$$
$$
C_3(P_3) = 0.25 P_3^2 + 40 P_3 + 1049
$$

Sujeito à restrição de balanço de potência (igualdade):

$$
P_1 + P_2 + P_3 = 850 + P_L
$$

Onde $P_L$ é a perda na transmissão:

$$
P_L = \sum_{i=1}^3 \sum_{j=1}^3 P_i B_{ij} P_j
$$

Com a matriz de coeficientes de perda $B$:

$$
B = \begin{bmatrix}
0.000049 & 0.000014 & 0.000015 \\
0.000014 & 0.000045 & 0.000016 \\
0.000015 & 0.000016 & 0.000039
\end{bmatrix}
$$

E as restrições de capacidade (desigualdades):

$$
150 \leq P_1 \leq 600
$$
$$
100 \leq P_2 \leq 400
$$
$$
50 \leq P_3 \leq 200
$$

In [ ]:
import numpy as np
from otimo import *

# Definição da Matriz 
B = np.array([
    [0.000049, 0.000014, 0.000015],
    [0.000014, 0.000045, 0.000016],
    [0.000015, 0.000016, 0.000039]
])

# Função Objetivo
def objective(x):
    P1, P2, P3 = x
    C1 = 0.15 * P1**2 + 38 * P1 + 756
    C2 = 0.1 * P2**2 + 46 * P2 + 451
    C3 = 0.25 * P3**2 + 40 * P3 + 1049
    return C1 + C2 + C3

# Restrição de Igualdade (Balanço de Potência)
def h_balanco(x):
    P = np.array(x)
    PL = P.T @ B @ P
    # P1 + P2 + P3 = 850 + PL => P1 + P2 + P3 - PL - 850 = 0
    return np.sum(P) - PL - 850

# Restrições de Desigualdade (Limites de Capacidade)
# Forma: g(x) <= 0 (<) ou g(x) >= 0 (>)

# 150 <= P1 <= 600
def g_p1_min(x): return x[0] - 150 # >= 0
def g_p1_max(x): return x[0] - 600 # <= 0

# 100 <= P2 <= 400
def g_p2_min(x): return x[1] - 100 # >= 0
def g_p2_max(x): return x[1] - 400 # <= 0

# 50 <= P3 <= 200
def g_p3_min(x): return x[2] - 50 # >= 0
def g_p3_max(x): return x[2] - 200 # <= 0

restrictions = [
    h_balanco,
    g_p1_min, g_p1_max,
    g_p2_min, g_p2_max,
    g_p3_min, g_p3_max
]

tipos_de_restricoes = [
    '=',
    '>', '<',
    '>', '<',
    '>', '<'
]

# Configuração dos Métodos
x0 = np.array([600.0, 400.0, 200.0]) # Initial guess
busca_1d = SecaoAurea(precisao=1e-6)
irrestrito = Gradiente(busca_1d, precisao=1e-6)

metodos = {
    'Penalidade Exterior': PenalidadeExterior(),
    'Lagrangeano Aumentado': LagrangeanoAumentado(),
}

for nome, metodo in metodos.items():
    print(f"\n--- Método: {nome} ---")
    resultado = metodo.resolva(objective, x0, restrictions, tipos_de_restricoes, irrestrito)
    print(f"Solução ótima encontrada:")
    print(f"P1 = {resultado.x[0]:.4f} MW")
    print(f"P2 = {resultado.x[1]:.4f} MW")
    print(f"P3 = {resultado.x[2]:.4f} MW")
    print(f"Resultado: x = {resultado.x}")
    print(f"Iterações = {resultado.iter}")
    #print(f"Avaliações de Função = {resultado.aval}")

        



--- Método: Penalidade Exterior ---
Solução ótima encontrada:
P1 = 294.8470 MW
P2 = 399.2919 MW
P3 = 175.5602 MW
Resultado: x = [294.84698495 399.29187579 175.56016393]
Iterações = 20

--- Método: Lagrangeano Aumentado ---
Solução ótima encontrada:
P1 = 294.9242 MW
P2 = 399.2341 MW
P3 = 175.5714 MW
Resultado: x = [294.9241675  399.23405389 175.57140164]
Iterações = 19
